<a href="https://colab.research.google.com/github/ShreejayShakya28/ASR-LLM-Pipeline/blob/feat%2FUse-Requirements.txt/RAG/notebook/RAG_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cell 0 : Import Drive

In [1]:
# Mount Google Drive — run this first every session
from google.colab import drive
drive.mount('/drive')

import os
os.makedirs('/drive/MyDrive/nepal_rag_index', exist_ok=True)
print("✅ Drive mounted — index will persist at /drive/MyDrive/nepal_rag_index")

Mounted at /drive
✅ Drive mounted — index will persist at /drive/MyDrive/nepal_rag_index


## Cell 2 — Clone Repo

In [2]:
import os

# ── Config — change this line only ───────────────────────────
BRANCH = "feat/Use-Requirements.txt"        # "main" | "develop" | any branch name
# ─────────────────────────────────────────────────────────────

REPO_URL = "https://github.com/ShreejayShakya28/ASR-LLM-Pipeline.git"
REPO_DIR = "/content/ASR-LLM-Pipeline"
RAG_DIR  = f"{REPO_DIR}/RAG"

if os.path.exists(REPO_DIR):
    !git -C {REPO_DIR} fetch --all
    !git -C {REPO_DIR} checkout {BRANCH}
    !git -C {REPO_DIR} pull origin {BRANCH}
    print(f"✅ Repo updated — branch: {BRANCH}")
else:
    !git clone -b {BRANCH} {REPO_URL} {REPO_DIR}
    print(f"✅ Repo cloned — branch: {BRANCH}")

import sys
if RAG_DIR not in sys.path:
    sys.path.insert(0, RAG_DIR)
print(f"✅ sys.path → {RAG_DIR}")

Cloning into '/content/ASR-LLM-Pipeline'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (63/63), done.
Receiving objects: 100% (75/75), 86.82 KiB | 773.00 KiB/s, done.
remote: Total 75 (delta 14), reused 50 (delta 6), pack-reused 0 (from 0)
Resolving deltas: 100% (14/14), done.
✅ Repo cloned — branch: feat/Use-Requirements.txt
✅ sys.path → /content/ASR-LLM-Pipeline/RAG


In [3]:
# Cell 1 — Install
!pip install -q -r /content/ASR-LLM-Pipeline/RAG/requirements.txt
print("✅ Dependencies installed")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 27.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.2/306.2 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 117.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 14.5 MB/s eta 0:00:00
✅ Dependencies installed


## Cell 3 — Load Models
*Takes ~2 min on first run. Weights are cached in `/root/.cache` for the session.*

In [4]:
from rag.models import embedding_model, reranker, tokenizer, llm
print("\n✅ All models ready — proceed to refresh.")

📦 Loading embedding model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/558 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

   ✅ google/flan-t5-large

✅ All models ready.

✅ All models ready — proceed to refresh.


## Cell 4 — Daily Refresh
*Run once per session. Scrapes new articles, skips already-indexed URLs.*

> **No background scheduler** — Colab disconnects too fast. Just run this on connect.

In [5]:
from rag.pipeline import daily_refresh

daily_refresh()
# To override limits:
# daily_refresh(max_per_feed=30)

✅ indic-nlp-library loaded (Nepali tokenization active)
🗓️  Daily Refresh — 2026-02-23 15:30
✅ DB ready: /drive/MyDrive/nepal_rag_index/metadata.db
🔍 Testing feeds...

   ✅  20 entries | https://english.onlinekhabar.com/feed
   ✅  20 entries | https://www.spotlightnepal.com/feed/
   ✅  10 entries | https://nepaleconomicforum.org/feed
   ✅  11 entries | https://techmandu.com/feed
   ✅  12 entries | https://nepalnews.com/feed
   ✅  12 entries | https://newsofnepal.com/feed/
   ❌   0 status  | https://www.recordnepal.com/feed
   ❌   0 status  | https://nepalipost.com/feed
   ❌ 301 status  | https://www.nepalitimes.com/feed
   ❌ 404 status  | https://myrepublica.nagariknetwork.com/feed
   ❌ 200 status  | https://kathmandupost.com/feed
   ❌ 404 status  | https://thehimalayantimes.com/feed
   ✅  55 entries | https://onlinekhabar.com/feed
   ✅   5 entries | https://setopati.com/feed
   ❌ 301 status  | https://ratopati.com/feed
   ❌ 200 status  | https://ekantipur.com/rss
   ✅  20 entries | ht

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

✅ FAISS: 590 total vectors | SQLite: +106 chunks added

🎉 Done! +66 articles | +106 chunks


## Cell 5 — Ask Questions

In [6]:
from rag.inference import ask

ask("Recent Nepali flood causality?")

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



🔍 Recent Nepali flood causality?

📰 Top 2 sources:
   [-1.14] Anticipatory Action and Social Protection for Climate Resilience in Nepal (2026-01-26)  cosine=0.5891
   [-9.27] Beyond Promises – The Need for Strategy to Deliver Economic Transformation (2026-02-11)  cosine=0.4549

🤖 Generating answer…

💬 Climate change is having significant impact in Nepal and disasters are becoming more frequent and intense over the course of this century. Accelerated glacial melt, more erratic monsoons, and rising floods, landslides, and droughts are disrupting lives and livelihoods. These shocks are undermining hard-won development gains and increasing landlessness, food insecurity, multidimensional poverty, and displacement.

📚 Sources:
   [1] Anticipatory Action and Social Protection for Climate Resilience in Nepal
        https://nepaleconomicforum.org/anticipatory-action-and-social-protection-for-climate-resilience-in-nepal/
        2026-01-26 | rerank=-1.1352
   [2] Beyond Promises – The Need for

'Climate change is having significant impact in Nepal and disasters are becoming more frequent and intense over the course of this century. Accelerated glacial melt, more erratic monsoons, and rising floods, landslides, and droughts are disrupting lives and livelihoods. These shocks are undermining hard-won development gains and increasing landlessness, food insecurity, multidimensional poverty, and displacement.'

In [7]:
from rag.inference import ask

ask("who is running from Rupendehi-2 for elections")


🔍 who is running from Rupendehi-2 for elections

❌ No relevant articles found in the last 30 days.
   Try: lower min_cosine (e.g. 0.35) or increase days_filter (e.g. 60).


In [8]:
ask("What is Prime Minister Karki doing?")


🔍 What is Prime Minister Karki doing?

📰 Top 3 sources:
   [+6.91] Prime Minister Directs Security Officials: “Ensure Free and Peaceful Elections” (2026-02-23)  cosine=0.5644
   [+6.86] Prime Minister Karki in Mustang (2026-02-20)  cosine=0.5879
   [+6.02] Prime Minister Directs Tightening of Election Security Arrangements (2026-02-23)  cosine=0.6008

🤖 Generating answer…

💬 Prime Minister Sushila Karki has directed security officials in Lumbini Province to ensure that the House of Representatives elections are conducted in a free and peaceful manner.

📚 Sources:
   [1] Prime Minister Directs Security Officials: “Ensure Free and Peaceful Elections”
        https://www.spotlightnepal.com/2026/02/22/prime-minister-directs-security-officials-ensure-free-and-peaceful-elections/
        2026-02-23 | rerank=6.9119
   [2] Prime Minister Karki in Mustang
        https://english.onlinekhabar.com/prime-minister-karki-in-mustang.html
        2026-02-20 | rerank=6.8607
   [3] Prime Minister Direc

'Prime Minister Sushila Karki has directed security officials in Lumbini Province to ensure that the House of Representatives elections are conducted in a free and peaceful manner.'

In [9]:
ask("What is the economic situation in Nepal?")


🔍 What is the economic situation in Nepal?

📰 Top 3 sources:
   [+2.01] Can Nepal’s Entrepreneurs Bootstrap Their Businesses?: Insights from the Life-Cycle Deficit (2026-02-16)  cosine=0.5923
   [+1.01] Nepal–India Economic Cooperation Should Be Made More Practical and Result-Oriented: President Dhakal (2026-02-23)  cosine=0.5582
   [+0.89] Understanding Inequality Beyond Income: Structural Inequality, Institutions, and Protest (2026-02-02)  cosine=0.4899

🤖 Generating answer…

💬 The report provides an assessment of age-specific consumption and labor income patterns of Nepali citizens and, in doing so, offers critical insights into how economic resources are generated and redistributed across generations in Nepal.

📚 Sources:
   [1] Can Nepal’s Entrepreneurs Bootstrap Their Businesses?: Insights from the Life-Cycle Deficit
        https://nepaleconomicforum.org/can-nepals-entrepreneurs-bootstrap-their-businesses-insights-from-the-life-cycle-deficit/
        2026-02-16 | rerank=2.012
 

'The report provides an assessment of age-specific consumption and labor income patterns of Nepali citizens and, in doing so, offers critical insights into how economic resources are generated and redistributed across generations in Nepal.'

In [10]:
# Nepali (Devanagari) works too
ask("नेपालमा के भइरहेको छ?")


🔍 नेपालमा के भइरहेको छ?

📰 Top 3 sources:
   [+8.90] एक-दुई नेताको बोलीको ‘सरस्वती’ बिग्रिँदा मैले जनतासमक्ष क्षमा मागेँ (2026-02-23)  cosine=0.5163
   [+8.86] पोखराबाट पूर्ण क्षमतामा यात्रु बोकेर बस हिडेको थियो (2026-02-23)  cosine=0.5478
   [+8.64] भैंसेगौडा दुर्घटना:  सामग्रीको अभावले पाँच घण्टापछि मात्र घाइतेको उद्धार (2026-02-23)  cosine=0.6221

🤖 Generating answer…

💬 

📚 Sources:
   [1] एक-दुई नेताको बोलीको ‘सरस्वती’ बिग्रिँदा मैले जनतासमक्ष क्षमा मागेँ
        https://www.onlinekhabar.com/2026/02/1873716/i-apologized-to-the-people-when-the-saraswati-of-one-or-two-leaders-speech-was-spoiled
        2026-02-23 | rerank=8.8951
   [2] पोखराबाट पूर्ण क्षमतामा यात्रु बोकेर बस हिडेको थियो
        https://nagariknews.nagariknetwork.com/social-affairs/the-bus-had-left-pokhara-carrying-passengers-at-full-capacity-81-85.html
        2026-02-23 | rerank=8.8633
   [3] भैंसेगौडा दुर्घटना:  सामग्रीको अभावले पाँच घण्टापछि मात्र घाइतेको उद्धार
        https://newsofnepal.com/2026/02/23/755488/

''

## Cell 6 — Tune Retrieval (optional)

| Symptom | Fix |
|---|---|
| No results | Lower `min_cosine` or raise `days_filter` |
| Off-topic answers | Raise `min_cosine` or lower `days_filter` |
| Missing older news | Raise `days_filter` |

In [11]:
# Widen the net — good for less-covered topics
ask("Nepal foreign policy", min_cosine=0.35, days_filter=60)


🔍 Nepal foreign policy

📰 Top 3 sources:
   [+4.20] Nepal–Portugal Bilateral Consultation Mechanism Meeting Concludes in Lisbon (2026-02-23)  cosine=0.5046
   [+3.18] Nepal–India Economic Cooperation Should Be Made More Practical and Result-Oriented: President Dhakal (2026-02-23)  cosine=0.568
   [+2.34] Beyond Promises – The Need for Strategy to Deliver Economic Transformation (2026-02-11)  cosine=0.5744

🤖 Generating answer…

💬 The first meeting of the Nepal Portugal Bilateral Consultation Mechanism (BCM) has been held in Lisbon, the capital of Portugal.

📚 Sources:
   [1] Nepal–Portugal Bilateral Consultation Mechanism Meeting Concludes in Lisbon
        https://www.spotlightnepal.com/2026/02/21/nepalportugal-bilateral-consultation-mechanism-meeting-concludes-in-lisbon/
        2026-02-23 | rerank=4.198
   [2] Nepal–India Economic Cooperation Should Be Made More Practical and Result-Oriented: President Dhakal
        https://www.spotlightnepal.com/2026/02/21/nepalindia-economic-coo

'The first meeting of the Nepal Portugal Bilateral Consultation Mechanism (BCM) has been held in Lisbon, the capital of Portugal.'

In [12]:
# Tighten for very recent, specific news
ask("Kathmandu flood", min_cosine=0.25, days_filter=7)


🔍 Kathmandu flood

📰 Top 3 sources:
   [-1.02] Bus plunges into Trishuli River, 17 dead and 27 injured (2026-02-23)  cosine=0.5115
   [-2.62] Trishuli Bus Accident Update: Death Toll Reaches 18 (2026-02-23)  cosine=0.4261
   [-2.98] Death toll in Dhading bus crash climbs to 19; Government forms probe panel (2026-02-23)  cosine=0.4126

🤖 Generating answer…

💬 A passenger bus travelling from Pokhara to Kathmandu plunged into the Trishuli River, killing six women and eleven men.

📚 Sources:
   [1] Bus plunges into Trishuli River, 17 dead and 27 injured
        https://english.onlinekhabar.com/bus-plunges-into-trishuli-river.html
        2026-02-23 | rerank=-1.0151
   [2] Trishuli Bus Accident Update: Death Toll Reaches 18
        https://www.spotlightnepal.com/2026/02/23/trishuli-bus-accident-update-death-toll-reaches-18/
        2026-02-23 | rerank=-2.6249
   [3] Death toll in Dhading bus crash climbs to 19; Government forms probe panel
        https://english.onlinekhabar.com/death-tol

'A passenger bus travelling from Pokhara to Kathmandu plunged into the Trishuli River, killing six women and eleven men.'

In [13]:
# More sources in context (default top_k=3)
ask("Nepal economy", top_k=5)


🔍 Nepal economy

📰 Top 5 sources:
   [+3.77] Nepal–India Economic Cooperation Should Be Made More Practical and Result-Oriented: President Dhakal (2026-02-23)  cosine=0.6508
   [+2.94] Nepal Clears Path to Receive Additional NPR 6 Billion in Financial Support from the IMF (2026-02-23)  cosine=0.4501
   [+2.71] Can Nepal’s Entrepreneurs Bootstrap Their Businesses?: Insights from the Life-Cycle Deficit (2026-02-16)  cosine=0.6417
   [+2.60] Re-Globalization: Riding the Waves of Global Change (2026-01-27)  cosine=0.5973
   [+0.86] Understanding Inequality Beyond Income: Structural Inequality, Institutions, and Protest (2026-02-02)  cosine=0.497

🤖 Generating answer…

💬 President Chandra Prasad Dhakal has emphasized the need to make Nepal-India economic cooperation more result-oriented.

📚 Sources:
   [1] Nepal–India Economic Cooperation Should Be Made More Practical and Result-Oriented: President Dhakal
        https://www.spotlightnepal.com/2026/02/21/nepalindia-economic-cooperation-sho

'President Chandra Prasad Dhakal has emphasized the need to make Nepal-India economic cooperation more result-oriented.'

## Cell 7 — Feed Management (optional)
Check which feeds are alive, or add a one-off feed without editing `config.py`.

In [14]:
from rag.scraper import test_feeds
from rag.config  import ALL_CANDIDATE_FEEDS

working_feeds = test_feeds(ALL_CANDIDATE_FEEDS)
print(f"\n{len(working_feeds)} feeds active")

🔍 Testing feeds...

   ✅  20 entries | https://english.onlinekhabar.com/feed
   ✅  20 entries | https://www.spotlightnepal.com/feed/
   ✅  10 entries | https://nepaleconomicforum.org/feed
   ✅  11 entries | https://techmandu.com/feed
   ✅  12 entries | https://nepalnews.com/feed
   ✅  12 entries | https://newsofnepal.com/feed/
   ❌   0 status  | https://www.recordnepal.com/feed
   ❌   0 status  | https://nepalipost.com/feed
   ❌ 301 status  | https://www.nepalitimes.com/feed
   ❌ 404 status  | https://myrepublica.nagariknetwork.com/feed
   ❌ 200 status  | https://kathmandupost.com/feed
   ❌ 404 status  | https://thehimalayantimes.com/feed
   ✅  55 entries | https://onlinekhabar.com/feed
   ✅   5 entries | https://setopati.com/feed
   ❌ 301 status  | https://ratopati.com/feed
   ❌ 200 status  | https://ekantipur.com/rss
   ✅  20 entries | https://nagariknews.nagariknetwork.com/feed
   ✅  20 entries | https://arthasarokar.com/feed

✅ 10 working feeds / 18 tested

10 feeds active


In [15]:
# Add a feed just for this session
extra_feeds = working_feeds + ["https://some-new-feed.com/rss"]
daily_refresh(feed_urls=extra_feeds)

🗓️  Daily Refresh — 2026-02-23 15:36
✅ DB ready: /drive/MyDrive/nepal_rag_index/metadata.db

📦 369 URLs already indexed — will skip these.


📡 https://english.onlinekhabar.com/feed
   📊 0 ok | 0 failed

📡 https://www.spotlightnepal.com/feed/
   📊 0 ok | 0 failed

📡 https://nepaleconomicforum.org/feed
   📊 0 ok | 0 failed

📡 https://techmandu.com/feed
   📊 0 ok | 0 failed

📡 https://nepalnews.com/feed
   → कांग्रेसले जिते मात्रै धर्म-संस्कृति जोगिन्छ : शेखर कोइराला…
      ♻️  BS4 fallback used
      ✅ 134 words
   📊 1 ok | 0 failed

📡 https://newsofnepal.com/feed/
   📊 0 ok | 0 failed

📡 https://onlinekhabar.com/feed
   → कांग्रेसले सुर्खेत, पोखरा र काठमाडौंमा चुनावी सभा गर्ने…
      ❌ Both extractors failed
   → सडक दुर्घटना कम गर्न मुख्य दलका योजना : प्राधिकरण बनाउनेदेखि जीपी…
      ♻️  BS4 fallback used
      ✅ 548 words
   → सडकमा नागरिकको ज्यान जान्छ, समितिका प्रतिवेदन दराजमा थन्किन्छन्…
      ✅ 907 words
   → एक्स्ट्रिम एनर्जीले साम्बाको उपचारका लागि ५ लाख रुपैयाँ सहयोग गर्…
     

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ FAISS: 617 total vectors | SQLite: +27 chunks added

🎉 Done! +21 articles | +27 chunks


## Cell 8 — Index Stats (optional)
Confirm how much is in the index.

In [16]:
import sqlite3, faiss
from rag.config import DB_PATH, INDEX_PATH

conn   = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
cursor.execute("SELECT COUNT(*) FROM chunks")
n_chunks = cursor.fetchone()[0]
cursor.execute("SELECT COUNT(DISTINCT url) FROM chunks")
n_articles = cursor.fetchone()[0]
cursor.execute("SELECT MIN(date), MAX(date) FROM chunks")
date_range = cursor.fetchone()
conn.close()

index = faiss.read_index(INDEX_PATH)

print(f"📊 Index stats")
print(f"   Articles  : {n_articles}")
print(f"   Chunks    : {n_chunks}")
print(f"   Vectors   : {index.ntotal}")
print(f"   Date range: {date_range[0]} → {date_range[1]}")

📊 Index stats
   Articles  : 390
   Chunks    : 617
   Vectors   : 617
   Date range: 2026-01-19 → 2026-02-23
